In [1]:
### Paquetes usados para la validación de datos

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import unidecode
import libreria.data_management as dm
from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')

from libreria import login # Lógica para el inicio de sesión a las fuentes de datos

# Cargue de las conexiones a las fuentes de datos
cnxn_sandbox, cursor_sandbox = login.login_sql(login.tipo_bodega.SANDBOX)
cnxn_reports, cursor_reports = login.login_sql(login.tipo_bodega.REPORTS)

In [2]:
# SECOP II
pd.read_sql_query("SELECT TOP 10 * FROM CCE_Reports.SECOPII.V_HistoricoProcedimientos_Depurado", cnxn_reports)

,CompanyName,BuyerVat,PCICode,ProcedureRequestID,Reference,PPI,BuyerDossierID,ProcedureRequestName,ProcedureRequestDescription2,ProfileDocUniqueName,PhaseName,RequestOnlinePublishingDate,Fecha Publicación,Fecha Publicacion fase planeacion precalficacion,Fecha Publicacion fase seleccion precalficacion,Fecha de Menifestacion de Interes,Fecha de publicacion de la fase borrador,Fecha de Publicacion de la fase Seleccion,BasePrice,SubmissionUser,Market,RequestType,ProcedureRequestType,OpenByCategory,LegalFrameworkName,Duration,DurationType,DueDateForReceivingReplies,OpeningRepliesDate,EffectiveOpeningDate,BusinessOperationState,BusinessOperationName,BusinessOperationCountry,AllowAuction,SuppliersInvited,DirectSuppliersInvited,ProcedureVisualizationCount,SuppliersExpressedInterestCount,RepliesCount,ExternalRepliesCount,CounterProposalRepliesCount,DistinctSupplierWithReplies,NumLotes,ProcedureRequestDescription,ProcedureRequestState,ProcedureRequestStateId,Awarded,Award ID,AwardDate,AwardTotalPrice,AwarderDisplayName,SupplierName,SupplierVat,SupplierPhoneNumber,SupplierEmailAddress,DimMainCategoryCode,DimMainCategoryTypeCode,ProcedureRequestOpeningState,TypeOfContract,SubTypeOfContract,AddicionalCategoryCodes,Tipo,CodigoProveedor,ContractNoticeUniqueIdentifier,Departamento Entidad,Ciudad Entidad,Departamento Proveedor,Ciudad Proveedor,Codigo Moneda,NumeroAdendas,AdtEditedAt,OrdenEntidad,Justificación Modalidad de Contratación,Entidad Centralizada,URLProceso,Codigo Entidad,EstadoResumen


In [3]:
# SECOP II
pd.read_sql_query("SELECT TOP 10 * FROM CCE_Reports.SECOPII.V_HistoricoContratos_Depurado", cnxn_reports)

,Nombre Entidad,Código Entidad,NIT Entidad,Orden,Sector,Rama,State,Province,City,Entidad Centralizada,Departamento Entidad,Ciudad Entidad,TipoDocProveedor,Documento Proveedor,Nombre Proveedor,Es Grupo,Es Pyme,Departamento Proveedor,Ciudad Proveedor,RequestReference,Description,JustificationTypeOfContractCode,TypeContract,ContractReference,ContractStartDate,ContractEndDate,DeliverStartDate,DeliverEndDate,Condiciones Entrega,ContractState,AdvancePayments,AdvancePayIndependentProperty,Liquidación,Obligación Ambiental,Obligación de Consumo,Reversión,ContractValue,AdvancedPaymentValue,InvoicedValue,PendingPaymentValue,PaidValue,AmortizedValue,PendingAmortizationValue,PendingExecutionValue,DimMainCategoryCode,ProcedureProfileLabel,AprovalDate,ContractNoticeUniqueIdentifier,Codigo Moneda,Código BPIN,Año BPIN,Estado BPIN,Origen de los Recursos,Destino Gasto,ReferenciaContrato,EsPostConflicto,Saldo CDP,Saldo Vigencia,FechaModificacion,Días Adicionados,Puntos del Acuerdo,Pilares del Acuerdo,URLProceso,AprovalDateDate,Nombre Representante Legal,Nacionalidad Representante Legal,Domicilio Representante Legal,Tipo de Identificación Representante Legal,Identificación Representante Legal,Género Representante Legal,Presupuesto General de la Nacion – PGN,Sistema General de Participaciones,Sistema General de Regalías,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos de Credito,Recursos Propios,Codigo Proveedor,Fecha Inicio Liquidacion,Fecha Fin Liquidacion,Objeto Contrato
0,MINISTERIO DE CULTURA,700637010,830034348,Nacional,Cultura,Ejecutivo,None,Bogotá,Distrito Capital de Bogotá,Centralizada,No Definido,No Definido,NIT,9002592566,Corporación Cultural Atabaques,No,No,Distrito Capital de Bogotá,Bogotá,CO1.BDOS.862805,"Aunar recursos humanos, técnicos y financieros...",Decree092/2017,DecreeLaw092/2017,CO1.PCCNTR.1020206,2019-07-11 12:00:00,2019-12-10 23:59:00,2019-07-11 12:00:00,2019-12-10 23:59:00,Como acordado previamente,Activo,False,False,No,No,No,No,8.000000e+07,0.0,0.0,8.000000e+07,0.0,0.0,0.0,8.000000e+07,V1.80101604,Contratación régimen especial (con ofertas),2019-07-10 11:21:38.433,CO1.NTC.857401,COP,No Definido,2019,Válido,Presupuesto de entidad nacional,Inversión,2348 2019,No,80000000.000000,0.0,2019-07-11 02:05:00,0,None,None,https://community.secop.gov.co/Public/Tenderin...,2019-07-10,Wilfran Barrios Paz,Colombiano,Urbanización Villas de la Candelaria Mz 13 Lt 14,None,73.139.418,None,NaN,NaN,NaN,NaN,NaN,NaN,702729682,NaT,NaT,Aunar recursos humanos técnicos y financieros ...
1,COMANDO AEREO DE COMBATE No. 4,701571077,800141632,Nacional,defensa,Ejecutivo,None,Melgar,Tolima,Centralizada,Tolima,Melgar,NIT,901115039,Trabajos Rápidos SAS,No,No,Tolima,Espinal,CO1.BDOS.857504,MANTENIMIENTO CORRECTIVO Y PREVENTIVO EQUIPO A...,Presupuesto inferior al 10% de la menor cuantía,Prestación de servicios,CO1.PCCNTR.1020207,2019-07-03 08:00:00,2019-12-05 23:00:00,2019-07-03 08:00:00,2019-12-05 23:00:00,DAP - Entregado en un punto (lugar de destino ...,terminado,False,False,Si,No,No,No,1.500000e+07,0.0,15000000.0,0.000000e+00,15000000.0,0.0,0.0,0.000000e+00,V1.21101500,Mínima cuantía,2019-07-03 15:50:37.383,CO1.NTC.850850,COP,No Definido,2023,Válido,Presupuesto de entidad nacional,Funcionamiento,085-00-G-CACOM-4-GRUSE-2019,No,15000000.000000,0.0,2023-07-29 19:02:00,0,None,None,https://community.secop.gov.co/Public/Tenderin...,2019-07-03,Omar Ricardo Ortiz Sanabria,Colombiano,Bogotá,None,79388619,None,NaN,NaN,NaN,NaN,NaN,NaN,702331612,2020-04-01 13:00:00,2020-04-01 16:00:00,MANTENIMIENTO CORRECTIVO Y PREVENTIVO EQUIPO A...
2,INSTITUTO NACIONAL DE METROLOGIA,700656093,900494393,Nacional,No aplica/No pertenece,Ejecutivo,None,Bogotá,Distrito Capital de Bogotá,Centralizada,Antioquia,No Definido,NIT,800226923,Signal Marketing S.A.S.,No,Si,Antioquia,Itagui,CO1.BDOS.861658,“Contratar la elaboración y adquisición de mat...,Presupuesto inferior al 10% de la menor cuantía,Suministros,CO1.PCCNTR.1020208,2019-07-05 12:00:00,2019-12-31 12:

In [5]:
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023 ----
## El calculo se realiza para optener los contratos unicos tanto en valor como en cantidad.       
## Los datos corresponden con la información reportada por las entidades estatales en el SECOP II 
pd.read_sql_query("SELECT DISTINCT MONTH(VALOR.AprovalDate)	AS 'MES', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

,MES,CONTRATOS,VALOR
0,1,126273,1.295318e+14
1,2,144824,1.390086e+13
2,3,83906,1.501784e+14
3,4,44232,6.831870e+12
4,5,66548,7.496111e+12
5,6,103166,1.796781e+13


In [6]:
# CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,None,2802,1.036596e+11
1,CCE-20-Concurso_Meritos_Sin_Lista_Corta_1Sobre,675,6.297684e+11
2,Concurso de méritos abierto,5,6.054017e+09
3,Contratación directa,462300,2.898331e+14
4,Contratación Directa (con ofertas),5161,6.603256e+12
5,Contratación régimen especial,73187,9.977852e+12
6,Contratación régimen especial (con ofertas),3174,3.353274e+12
7,Enajenación de bienes con sobre cerrado,12,1.048216e+09
8,Enajenación de bienes con subasta,16,1.094314e+09
9,Licitación pública,805,4.377965e+12


In [7]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,Acuerdo Marco de Precios,149,0.000000e+00
1,Arrendamiento de inmuebles,3426,4.423872e+11
2,Arrendamiento de muebles,207,2.537853e+11
3,Asociación Público Privada,25,1.530821e+10
4,Comisión,11,3.166350e+10
5,Comodato,1545,1.509166e+11
6,Compraventa,4005,6.247504e+11
7,Concesión,6,7.766872e+11
8,Consultoría,401,1.245037e+11
9,DecreeLaw092/2017,18381,3.701815e+12


In [8]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

,ENTIDAD,CONTRATOS,VALOR
0,MUNICIPIOS ASOCIADOS DE LA SUBREGION DE EMBALS...,42,1.420791e+14
1,DISPENSARIO MEDICO ORIENTE,194,1.203634e+14
2,DISTRITO ESPECIAL DE CIENCIA TECNOLOGIA E INNO...,1209,3.110818e+12
3,INSTITUTO DE DESARROLLO URBANO,1586,1.837489e+12
4,DEPARTAMENTO DE ANTIOQUIA,1021,1.691273e+12
5,RNEC,262,1.178253e+12
6,CORPORACION AUTONOMA REGIONAL DE SANTANDER,982,1.172425e+12
7,UNIDAD NACIONAL DE PROTECCIÓN,1529,1.041562e+12
8,FONDO DE PASIVO SOCIAL DE FERROCARRILES NACION...,456,8.932025e+11
9,DEPARTAMENTO DE SANTANDER,3800,8.330502e+11


In [9]:
## DATOS DE LA ENTIDAD (Secretaría Distrital de Integración Social), CODIGO 702271321
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT MONTH(VALOR.AprovalDate)	AS 'MES', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

,MES,CONTRATOS,VALOR
0,1,1258,5.219759e+10
1,2,2630,3.897900e+11
2,3,1799,8.390169e+10
3,4,1052,9.617150e+10
4,5,808,3.306843e+10
5,6,800,1.452919e+11


In [10]:
## CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,None,1,4.699890e+07
1,Contratación directa,8205,6.790804e+11
2,Contratación régimen especial,7,3.730169e+10
3,Contratación régimen especial (con ofertas),121,7.837072e+10
4,Mínima cuantía,4,3.261550e+08
5,Selección Abreviada de Menor Cuantía,3,2.153714e+09
6,Selección abreviada subasta inversa,6,3.141430e+09


In [11]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,Arrendamiento de inmuebles,90,1.179757e+10
1,Comodato,2,0.000000e+00
2,DecreeLaw092/2017,121,7.837072e+10
3,No Especificado,1,4.699890e+07
4,Otro,39,4.263700e+11
5,Prestación de servicios,8092,2.817297e+11
6,Seguros,1,1.599632e+09
7,Suministros,1,5.064510e+08


In [12]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

,ENTIDAD,CONTRATOS,VALOR
0,(Secretaría Distrital de Integración Social),8347,8.004211e+11
